In [ ]:
# Custom methods to clean up the source code a bit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class cust():

    def __init__(self, dat):
        self.dat = dat


    def xy(self, x_dummies_list, X_list, y_list):

        X = self.dat[X_list]
        y = self.dat[y_list]

        # One hot encoding the pg column
        X = pd.get_dummies(X, columns = x_dummies_list, drop_first = True)
        # C_cols should be after get_dummies so we get ALL columns
        x_cols = X.columns
        # Convert the two into value arrays
        X = X.values
        y = y.values
        # We need y as a 1D array
        y = np.ravel(y)

        return X,y,x_cols


    # Same as xy except returns a dataframe instead of a float64 np array
    def xy_df(self, x_dummies_list, X_list, y_list):

        X = self.dat[X_list]
        y = self.dat[y_list]

        # One hot encoding the pg column
        X = pd.get_dummies(X, columns = x_dummies_list, drop_first = True)
        # C_cols should be after get_dummies so we get ALL columns
        x_cols = X.columns
        # We need y as a 1D array
        y = np.ravel(y)

        return X,y,x_cols


    def clean_data(self):
        self.dat.replace([np.inf, -np.inf], np.nan) # Replace inf
        self.dat = self.dat.dropna(axis=0, how = 'any') # Drop NA's on the rows axis
        # I kept getting a value error and this was the only thing that seemed to fix it
        self.dat = self.dat[~self.dat.isin([np.nan, np.inf, -np.inf]).any(1)]
        return self.dat


    def outlier_removal(self,var):
        IQR = self.dat[var].describe()['75%'] - self.dat[var].describe()['25%']
        min_val = self.dat[var].describe()['25%'] - (IQR * 1.5)
        max_val = self.dat[var].describe()['75%'] + (IQR * 1.5)

        self.dat = self.dat[(self.dat[var] > min_val) & (self.dat[var] < max_val)]
        plt.boxplot(self.dat[var])
        return self.dat

    @staticmethod
    def comparison_df(y_pred, y_test):
        # Dataframe of pred and actual y
        comparison_df = pd.DataFrame({'y_pred':y_pred, 'y_test':y_test})
        comparison_df['abs_difference'] = abs( comparison_df['y_pred'] - comparison_df['y_test'] )
        comparison_df['real_difference'] = comparison_df['y_pred'] - comparison_df['y_test']
        print(comparison_df.describe())
        # Show all sums
        print(comparison_df.sum())
        # Show average difference
        print ("Average Difference: ", comparison_df.sum()[2] / len(comparison_df))

        return comparison_df

In [ ]:
#%cd /content/drive/MyDrive/Prediction/Rangpur_Region

/content/drive/MyDrive/Prediction/Rangpur_Region


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
columns = ['Time', 'Node', 'Total(MB)']
data = pd.read_excel("Prediction_dataset.xlsx")
data.Time = pd.to_datetime(data.Time)
node_list = data["Node"]
nodes = node_list.unique()

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import LSTM_Prep_multivariate
from tensorflow.keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import ReduceLROnPlateau #Learning rate scheduler for when we reach plateaus
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100)

split = 1
sequence_length = 8
pred_days = 30 # predicted_vs_actual plot
EPOCHS = 50
validation = 0.2

# Reset model if we want to re-train with different splits
def reset_weights(model):
    import keras.backend as K
    session = K.get_session()
    for layer in model.layers:
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)
        if hasattr(layer, 'bias_initializer'):
            layer.bias.initializer.run(session=session)

# Instantiate the model
model = Sequential()

# Add the first layer.... the input shape is (Sample, seq_len-1, 1)
model.add(LSTM(
        input_shape = (sequence_length-1, 10), return_sequences = True,
        units = 100))

# Add the second layer.... the input shape is (Sample, seq_len-1, 1)
model.add(LSTM(
        input_shape = (sequence_length-1, 10),
        units = 100))

# Add the output layer, simply one unit
model.add(Dense(
        units = 3,
        activation = 'sigmoid'))
opt = optimizers.Adam(learning_rate=0.01)
model.compile(loss = 'mse', optimizer = 'adam')

model.save_weights('initializer_multivariate.h5')

import csv
f = open('excel_out_100_lstm_25_epo__multivariate.csv', 'w')
g = open('excel_out_100_lstm_25_epo__multivariate_kpi.csv', 'w')
writer = csv.writer(f)
writer2 = csv.writer(g)
date_written = False
from tqdm import tqdm
for node in tqdm(nodes):
    node_index = data["Node"] == node
    my_data = data[node_index]

    data_prep = LSTM_Prep_multivariate.Data_Prep(dataset = my_data)
    rnn_df, validation_df = data_prep.preprocess_rnn(date_colname = 'Time', numeric_colname = 'Total(GB)',features=['Total(GB)','AVG_NO_USER','MAX_NO_USER','WEEK_DAY'], pred_set_timesteps = pred_days)


    series_prep = LSTM_Prep_multivariate.Series_Prep(rnn_df,'Total(GB)','AVG_NO_USER','MAX_NO_USER','WEEK_DAY')
    window, X_min, X_max = series_prep.make_window(sequence_length = sequence_length,
                                                  train_test_split = split,
                                                  return_original_x = True)

    X_train, X_test, y_train, y_test = series_prep.reshape_window(window, train_test_split = split)
    # Epochs and validation split

    model.load_weights('initializer_multivariate.h5')


    # History object for plotting our model loss by epoch
    history = model.fit(X_train, y_train, epochs = EPOCHS, validation_split = validation,
              callbacks = [rlrop])
    # Loss History
    plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    #plt.show()

    row_data = [node]
    FuTure = Predict_Future(rnn_df, X_train, validation_df = validation_df, lstm_model = model)
    comparison_df, all_date, true_data, pred_data, metric_name, metric_val = FuTure.predicted_vs_actual(X_min = X_min, X_max = X_max, numeric_colname = 'Total(GB)')
    # pred_traffic, pred_date = future.predict_future(X_min = X_min, X_max = X_max, numeric_colname = 'TRAFFIC_MB', timesteps_to_predict = 7*24, return_future = True)
    if not date_written:
        writer.writerow(['eNodeB']+all_date)
        writer2.writerow(['eNodeB'] + metric_name)
    else:
        writer.writerow(['eNodeB'])
    writer.writerow(row_data + true_data)
    writer.writerow([''] + pred_data)
    writer.writerow([''] + metric_name)
    writer.writerow([''] + metric_val)
    writer2.writerow(row_data + metric_val)
    date_written = True
f.close()
g.close()

  0%|          | 0/100 [00:00<?, ?it/s]

Summary Statistics - ADF Test For Stationarity

ADF Stat is: -4.185661127017571.
P Val is: 0.0006967086191609373.
Critical Values (Significance Levels): 
1% : -3.494
5% : -2.889
10% : -2.582
Epoch 1/50
3/3 [==============================] - 4s 418ms/step - loss: 0.0275 - val_loss: 0.0746
Epoch 2/50
3/3 [==============================] - 0s 27ms/step - loss: 0.0226 - val_loss: 0.0707
Epoch 3/50
3/3 [==============================] - 0s 25ms/step - loss: 0.0222 - val_loss: 0.0731
Epoch 4/50
3/3 [==============================] - 0s 26ms/step - loss: 0.0227 - val_loss: 0.0736
Epoch 5/50
3/3 [==============================] - 0s 25ms/step - loss: 0.0219 - val_loss: 0.0763
Epoch 6/50
3/3 [==============================] - 0s 24ms/step - loss: 0.0217 - val_loss: 0.0743
Epoch 7/50
3/3 [==============================] - 0s 28ms/step - loss: 0.0218 - val_loss: 0.0722
Epoch 8/50
3/3 [==============================] - 0s 24ms/step - loss: 0.0216 - val_loss: 0.0714
Epoch 9/50
3/3 [================

  1%|          | 1/100 [00:13<21:54, 13.28s/it]

See Plot for predicted vs. actuals
2021-10-15 00:00:00
Summary Statistics - ADF Test For Stationarity

ADF Stat is: -3.6795555395367425.
P Val is: 0.004410088740979317.
Critical Values (Significance Levels): 
1% : -3.495
5% : -2.89
10% : -2.582
Epoch 1/50
3/3 [==============================] - 0s 39ms/step - loss: 0.0647 - val_loss: 0.0557
Epoch 2/50
3/3 [==============================] - 0s 27ms/step - loss: 0.0627 - val_loss: 0.0530
Epoch 3/50
3/3 [==============================] - 0s 26ms/step - loss: 0.0604 - val_loss: 0.0506
Epoch 4/50
3/3 [==============================] - 0s 26ms/step - loss: 0.0583 - val_loss: 0.0490
Epoch 5/50
3/3 [==============================] - 0s 29ms/step - loss: 0.0577 - val_loss: 0.0482
Epoch 6/50
3/3 [==============================] - 0s 26ms/step - loss: 0.0566 - val_loss: 0.0481
Epoch 7/50
3/3 [==============================] - 0s 27ms/step - loss: 0.0550 - val_loss: 0.0488
Epoch 8/50
3/3 [==============================] - 0s 31ms/step - loss: 0.053

  2%|▏         | 2/100 [00:20<15:46,  9.66s/it]

...

Summary Statistics - ADF Test For Stationarity

ADF Stat is: -3.103307428051175.
P Val is: 0.02630939026252593.
Critical Values (Significance Levels): 
1% : -3.495
5% : -2.89
10% : -2.582
Epoch 1/50
3/3 [==============================] - 0s 48ms/step - loss: 0.0395 - val_loss: 0.0450
Epoch 2/50
3/3 [==============================] - 0s 30ms/step - loss: 0.0338 - val_loss: 0.0439
Epoch 3/50
3/3 [==============================] - 0s 34ms/step - loss: 0.0269 - val_loss: 0.0451
Epoch 4/50
3/3 [==============================] - 0s 34ms/step - loss: 0.0238 - val_loss: 0.0500
Epoch 5/50
3/3 [==============================] - 0s 30ms/step - loss: 0.0246 - val_loss: 0.0541
Epoch 6/50
3/3 [==============================] - 0s 44ms/step - loss: 0.0251 - val_loss: 0.0517
Epoch 7/50
3/3 [==============================] - 0s 43ms/step - loss: 0.0238 - val_loss: 0.0470
Epoch 8/50
3/3 [==============================] - 0s 35ms/step - loss: 0.0228 - val_loss: 0.0439
Epoch 9/50
3/3 [====================

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
100%|██████████| 100/100 [11:29<00:00,  6.89s/it]

See Plot for predicted vs. actuals
2021-10-15 00:00:00
Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f9894d83cb0> (for post_execute):


ValueError: ignored

ValueError: ignored

In [ ]:
plt.plot(history.history['val_loss'])

In [ ]:
from numpy import newaxis
import numpy as np
class Predict_Future:

    def __init__(self, rnn_df, X_test, validation_df, lstm_model):
        self.rnn_df = rnn_df
        self.X_test = X_test
        self.validation_df = validation_df
        self.lstm_model = lstm_model

    def calc_metrics(self, true_y, predicted_y):
        metric_names = ['Forecast Bias', 'R-sqr', 'MAE', 'RMSE', 'MAPE']

        forecast_errors = [predicted_y[i]-true_y[i] for i in range(len(predicted_y))]
        bias = sum(forecast_errors) * 1.0/len(predicted_y)
        mean_absolute_error = np.mean( np.abs(forecast_errors) )
        root_mean_squared_error = np.sqrt(np.mean([number ** 2 for number in forecast_errors]))
        Unexplained_var = np.sum([number ** 2 for number in forecast_errors])
        mean_of_true = np.mean(true_y)
        Total_var = np.sum([(number-mean_of_true) ** 2 for number in true_y])
        r_sqr = 1 - Unexplained_var/Total_var
        percentage_error = [(predicted_y[i]-true_y[i])/true_y[i] if true_y[i]>0 else 0 for i in range(len(predicted_y))]
        mean_absolute_percentage_error = np.mean( np.abs(percentage_error) ) * 100

        metric_values = [bias, r_sqr, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error]

        return metric_names, metric_values


    def predicted_vs_actual(self, X_min, X_max, numeric_colname, only_future=False):
        # X_train, X_test, validation sequential. First prediction of validation will be from last of X_test
        # all true value is known, historical flag is of no use in this function
        curr_frame = self.X_test[len(self.X_test) - 1]
        curr_week_day = np.argmax(curr_frame[-1,3:]) # 3 are GB, AVG_USER and MAX_USER
        future = []

        for i in range(len(self.validation_df)):
            # append the prediction to our empty future list
            model_out = self.lstm_model.predict(curr_frame[newaxis, :, :])[0]
            future.append(model_out[0])
            # insert our predicted point to our current frame
            curr_week_day = np.remainder(curr_week_day+1, 7)
            week_mask = np.zeros((7))
            week_mask[curr_week_day] = 1
            curr_frame = np.insert(curr_frame, len(self.X_test[0]), np.concatenate((model_out,week_mask)), axis=0)
            # push the frame up one to make it progress into the future
            curr_frame = curr_frame[1:]

        def reverse_minmax(X, X_max=X_max, X_min=X_min):
            return X * (X_max - X_min) + X_min

        # Plot
        reverse_train_test_frame = pd.DataFrame(
            {numeric_colname: self.rnn_df[numeric_colname],
             "historical_flag": 1})
        #print('reverse_train_test_frame')
        #print(reverse_train_test_frame)

        reverse_future = pd.DataFrame({numeric_colname: [reverse_minmax(x) for x in future],
                                       "historical_flag": 0})

        reverse_curr_frame = pd.DataFrame(
            {numeric_colname: [x for x in self.validation_df[numeric_colname]],
             "historical_flag": 1})
        reverse_future = pd.DataFrame({numeric_colname: [reverse_minmax(x) for x in future],
                                       "historical_flag": 0})

        # Change the indicies! Only for FUTURE predictions
        if not only_future:
            reverse_future.index += len(reverse_train_test_frame)
            reverse_curr_frame.index += len(reverse_train_test_frame)

        print("See Plot for predicted vs. actuals")
        #plt.plot(reverse_curr_frame[numeric_colname])
        #plt.plot(reverse_future[numeric_colname])
        fig, ax = plt.subplots()
        fig.suptitle('Predicted Points Vs. Actuals (Validation)', fontsize=12)
        ax.plot(reverse_future[numeric_colname], color='red', label='Predicted')
        ax.plot(reverse_curr_frame[numeric_colname], color='green', label='Actual')
        ax.plot(reverse_train_test_frame[numeric_colname], color='blue', label='Train_test')
        import datetime
        day_cnt = reverse_train_test_frame.shape[0] + validation_df.shape[0]
        last_date = self.validation_df.index.array[-1]
        print(last_date)
        date_list = [(last_date - datetime.timedelta(days=x)).strftime("%d-%m-%y") for x in range(day_cnt)]

        #plt.xticks(range(self.validation_df.shape[0), self.validation_df.index.strftime("%d-%mmm-%y"), rotation='vertical')
        #plt.xticks(range(day_cnt),date_list[::-1], rotation='vertical')
        #ax.legend(loc='upper left')
        #plt.show()

        comparison_df = pd.DataFrame({"Validation": self.validation_df[numeric_colname],
                                      "Predicted": [reverse_minmax(x) for x in future]})
        #print("Validation Vs. Predicted")
        #print(comparison_df)
        true_data = reverse_train_test_frame[numeric_colname].tolist() + reverse_curr_frame[numeric_colname].tolist()
        predicted_data = ['']*reverse_train_test_frame.shape[0] + reverse_future[numeric_colname].tolist()
        metric_name, metric_val = self.calc_metrics(reverse_curr_frame[numeric_colname].tolist(), reverse_future[numeric_colname].tolist())
        return comparison_df, date_list[::-1], true_data, predicted_data, metric_name, metric_val

    def predict_future(self, X_min, X_max, numeric_colname, timesteps_to_predict, return_future=True):

        curr_frame = self.X_test[len(self.X_test) - 1]
        #curr_frame = self.X_test[0] # predic ton trained data
        future = []

        for i in range(timesteps_to_predict):
            # append the prediction to our empty future list
            future.append(self.lstm_model.predict(curr_frame[newaxis, :, :])[0, 0])
            # insert our predicted point to our current frame
            curr_frame = np.insert(curr_frame, len(self.X_test[0]), future[-1], axis=0)
            # push the frame up one to make it progress into the future
            curr_frame = curr_frame[1:]

        def reverse_minmax(X, X_max=X_max, X_min=X_min):
            return X * (X_max - X_min) + X_min

        # Reverse the original frame and the future frame
        reverse_curr_frame = pd.DataFrame(
            {numeric_colname: self.rnn_df[numeric_colname],
             "historical_flag": 1})
        reverse_future = pd.DataFrame({numeric_colname: [reverse_minmax(x) for x in future],
                                       "historical_flag": 0})

        # Change the indicies to show prediction next to the actuals in orange
        reverse_future.index += len(reverse_curr_frame)
        #reverse_future.index += 4

        print("See Plot for Future Predictions")
        plt.figure(figsize=(10, 6), dpi=80)
        plt.plot(reverse_curr_frame[numeric_colname],label='Training')
        plt.plot(reverse_future[numeric_colname],label='Predicted')
        plt.title("Predicted Future of " + str(timesteps_to_predict) + " days")
        import datetime
        day_cnt = self.rnn_df.shape[0] + reverse_future.shape[0]
        last_rnn_date = self.rnn_df.Time.array[-1]
        date_list = [(last_rnn_date - datetime.timedelta(days=x)).strftime("%d-%m-%y") for x in range(-reverse_future.shape[0],self.rnn_df.shape[0])]
        date_list_future = [(last_rnn_date - datetime.timedelta(days=x)).strftime("%d-%m-%y") for x in range(-reverse_future.shape[0],0)]
        plt.ylim(ymin=0)  # this line
        #date_list = [(last_rnn_date - datetime.timedelta(hours=x)).strftime("%d-%H") for x in range(-reverse_future.shape[0],self.rnn_df.shape[0])]
        #date_list_future = [(last_rnn_date - datetime.timedelta(hours=x)).strftime("%d-%m-%y %H:%M") for x in range(-reverse_future.shape[0],0)]
        #plt.ylim(ymin=0)  # this line
        #plt.xticks(range(self.validation_df.shape[0), self.validation_df.index.strftime("%d-%mmm-%y"), rotation='vertical')
        plt.xticks(range(day_cnt),date_list[::-1], rotation='vertical')
        #plt.xticks(range(0,day_cnt,12),date_list[::-12], rotation='vertical')
        plt.legend(loc='lower right')
        plt.show()

        if return_future:
            return reverse_future, date_list_future[::-1]